# A Python Notebook for loading Charging Session data from a network operator

Note: We are relying on datafiles that are not publicly available.

In [1]:
# Using pathlib for proper path management
from pathlib import Path

# Optimised data storage
import pyarrow.feather as feather

# Data manipulation
import pandas as pd
import geopandas as gpd
import numpy as np

# Data visualization
import seaborn as sns
import matplotlib.pyplot as plt
import plotly.express as px

# Regressions, stats and clustering
from sklearn.cluster import KMeans

### Charging sessions: Internal users
Dealing with Charging session data: loading, joining and persisting using Arrow

In [2]:
# Loading the data, joining service records and saving as feather
# This block need to be executed only once

# Note: we need openpyxl to read excel files
bbs = pd.read_excel(
    Path.joinpath(
        Path.cwd(), "data", "NETWORK_OPERATOR", "Charging sessions (BBS).xlsx"
    )
)
bbs_service_records = pd.read_excel(
    Path.joinpath(
        Path.cwd(),
        "data",
        "NETWORK_OPERATOR",
        "Charging sessions - Service Records (BBS).xlsx",
    )
)
bbs_joined = bbs.join(
    bbs_service_records.set_index("id"), on="servicerecord_ptr_id", how="inner"
)

feather.write_feather(
    bbs_joined,
    Path.joinpath(
        Path.cwd(),
        "data",
        "NETWORK_OPERATOR",
        "Charging sessions and service records (BBS).feather",
    ),
)

In [3]:
# Let's load the feather file
internal_users = feather.read_feather(
    Path.joinpath(
        Path.cwd(),
        "data",
        "NETWORK_OPERATOR",
        "Charging sessions and service records (BBS).feather",
    )
)
internal_users["internal_user"] = True
internal_users.info()

<class 'pandas.core.frame.DataFrame'>
Index: 52692 entries, 0 to 52691
Data columns (total 28 columns):
 #   Column                Non-Null Count  Dtype         
---  ------                --------------  -----         
 0   zip code              52580 non-null  object        
 1   town                  52580 non-null  object        
 2   country               52692 non-null  object        
 3   latitude              41130 non-null  float64       
 4   longitude             41130 non-null  float64       
 5   evse_id               52692 non-null  object        
 6   connector_type        52015 non-null  object        
 7   end_time              52692 non-null  datetime64[ns]
 8   kWh                   52692 non-null  float64       
 9   operator_name         52630 non-null  object        
 10  power_type            52588 non-null  object        
 11  remuneration_type     52692 non-null  object        
 12  servicerecord_ptr_id  52692 non-null  int64         
 13  user_tariff_fixed    

Performing spatial clipping with high-resolution Belgium boundary

Saving the result as a feather document

In [4]:
# Getting our boundary
belgium_gdf = gpd.read_file(
    Path.joinpath(
        Path.cwd(),
        "data",
        "EUROSTAT-ref-countries-2020-01m-shp",
        "CNTR_RG_01M_2020_4326",
        "CNTR_RG_01M_2020_4326.shp",
    )
)

# Making sure that the CRS and projection are okay
belgium_gdf = belgium_gdf.to_crs("EPSG:4326")
# We need only Belgium from the dataset
belgium_gdf = belgium_gdf[belgium_gdf["CNTR_ID"] == "BE"]

# Constructing a geodataframe from the internal users dataframe
internal_users_gdf = gpd.GeoDataFrame(
    data=internal_users,
    geometry=gpd.points_from_xy(internal_users.longitude, internal_users.latitude),
    crs="EPSG:4326",
)

# Let's check the shape of the geodataframe before clipping
print(internal_users_gdf.shape)

# Clipping with the border shape
internal_users_gdf = internal_users_gdf.clip(belgium_gdf)

# After clipping as well
print(internal_users_gdf.shape)

# Saving the clipped geodataframe as feather
feather.write_feather(
    pd.DataFrame(internal_users_gdf.drop(columns="geometry")),
    Path.joinpath(
        Path.cwd(),
        "data",
        "NETWORK_OPERATOR",
        "Charging sessions and service records (BBS) BE only.feather",
    ),
)

(52692, 29)
(39256, 29)


In [5]:
# A quick look at the dataframe
# internal_users.sample(5)

### Charging sessions: External users

Charging sessions of other users (using a non-network charging card) are listed in another dataset.

Loading, joining and persisting using Arrow

In [6]:
# Note: we need openpyxl to read excel files
abs = pd.read_excel(
    Path.joinpath(
        Path.cwd(), "data", "NETWORK_OPERATOR", "Charging sessions (ABS).xlsx"
    )
)

feather.write_feather(
    abs,
    Path.joinpath(
        Path.cwd(), "data", "NETWORK_OPERATOR", "Charging sessions (ABS).feather"
    ),
)

In [7]:
external_users = feather.read_feather(
    Path.joinpath(
        Path.cwd(), "data", "NETWORK_OPERATOR", "Charging sessions (ABS).feather"
    )
)
external_users.info()

<class 'pandas.core.frame.DataFrame'>
RangeIndex: 130195 entries, 0 to 130194
Data columns (total 21 columns):
 #   Column                 Non-Null Count   Dtype         
---  ------                 --------------   -----         
 0   id                     130195 non-null  int64         
 1   zip code               130147 non-null  object        
 2   town                   130147 non-null  object        
 3   country                130195 non-null  object        
 4   latitude               130195 non-null  float64       
 5   longitude              130195 non-null  float64       
 6   evse_id                130195 non-null  int64         
 7   connector_type         130195 non-null  object        
 8   time_started_session   130195 non-null  datetime64[ns]
 9   time_stopped_session   130195 non-null  datetime64[ns]
 10  start_kilo_watt_hour   130195 non-null  float64       
 11  final_kilo_watt_hour   130195 non-null  float64       
 12  remuneration_type      130195 non-null  obje

In [8]:
# Getting our boundary
belgium_gdf = gpd.read_file(
    Path.joinpath(
        Path.cwd(),
        "data",
        "EUROSTAT-ref-countries-2020-01m-shp",
        "CNTR_RG_01M_2020_4326",
        "CNTR_RG_01M_2020_4326.shp",
    )
)

# Making sure that the CRS and projection are okay
belgium_gdf = belgium_gdf.to_crs("EPSG:4326")
# We need only Belgium from the dataset
belgium_gdf = belgium_gdf[belgium_gdf["CNTR_ID"] == "BE"]

# Constructing a geodataframe from the external users dataframe
external_users_gdf = gpd.GeoDataFrame(
    data=external_users,
    geometry=gpd.points_from_xy(external_users.longitude, external_users.latitude),
    crs="EPSG:4326",
)

# Let's check the shape of the geodataframe before clipping
print(external_users_gdf.shape)

# Clipping with the border shape
external_users_gdf = external_users_gdf.clip(belgium_gdf)

# After clipping as well
print(external_users_gdf.shape)

# Saving the clipped geodataframe as feather
feather.write_feather(
    pd.DataFrame(external_users_gdf.drop(columns="geometry")),
    Path.joinpath(
        Path.cwd(),
        "data",
        "NETWORK_OPERATOR",
        "Charging sessions (ABS) BE only.feather",
    ),
)

(130195, 22)
(129494, 22)
